In [3]:
import os
import tensorflow as tf
import pandas as pd

In [4]:
import numpy as np
import shutil
import matplotlib
import matplotlib.pyplot as plt


In [5]:
pwd

'C:\\Users\\Kevin Tang\\Documents\\HackTheNorth2023\\AI'

In [6]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 1024

In [9]:
DATA_PATH = "../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_64dc6cae-932e-413a-bba5-831aca573647/data.csv"
TEST_PATH = "dataset/test_data.csv"

In [10]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]])
df.at[0, 2]

3

In [11]:
df = pd.read_csv(DATA_PATH)
print(df)

      age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0    75.0        0                       582         0                 20   
1    55.0        0                      7861         0                 38   
2    65.0        0                       146         0                 20   
3    50.0        1                       111         0                 20   
4    65.0        1                       160         1                 20   
..    ...      ...                       ...       ...                ...   
294  62.0        0                        61         1                 38   
295  55.0        0                      1820         0                 38   
296  45.0        0                      2060         1                 60   
297  45.0        0                      2413         0                 38   
298  50.0        0                       196         0                 45   

     high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \

In [25]:
df = pd.read_csv(DATA_PATH)

df = df.drop('BookingID', axis=1) # axis: 0 for row, 1 for column

print(len(df))

for i in range(len(df)):
    # ARRIVAL YEAR
    df.at[i, "ArrivalYear"] = df.at[i, "ArrivalYear"] % 2000

    # MEAL PARSING
    if df.at[i, "MealPlan"].startswith("Not"):
        df.at[i, "MealPlan"] = 0
    else:
        df.at[i, "MealPlan"] = df.at[i, "MealPlan"].split()[-1]

    # ROOM TYPE
    df.at[i, "RoomType"] = df.at[i, "RoomType"].split()[-1]

    # MARKET SEGMENT
    if df.at[i, "MarketSegment"] == "Online":
        df.at[i, "MarketSegment"] = 5
    elif df.at[i, "MarketSegment"] == "Offline":
        df.at[i, "MarketSegment"] = 4
    elif df.at[i, "MarketSegment"] == "Corporate":
        df.at[i, "MarketSegment"] = 3
    elif df.at[i, "MarketSegment"] == "Complementary":
        df.at[i, "MarketSegment"] = 2
    elif df.at[i, "MarketSegment"] == "Aviation":
        df.at[i, "MarketSegment"] = 1
    else:
        print("Recheck data.")

    # BOOKING STATUS
    if df.at[i, "BookingStatus"] == "Canceled":
        df.at[i, "BookingStatus"] = 0
    elif df.at[i, "BookingStatus"] == "Not_Canceled":
        df.at[i, "BookingStatus"] = 1

# BOOKING STATUS
target = df.pop("BookingStatus")
df

29020


,LeadTime,ArrivalYear,ArrivalMonth,ArrivalDate,NumWeekendNights,NumWeekNights,MealPlan,Parking,RoomType,NumAdults,NumChildren,MarketSegment,RepeatedGuest,NumPrevCancellations,NumPreviousNonCancelled,AvgRoomPrice,SpecialRequests
0,10,18,3,31,0,1,1,0,1,1,0,3,0,0,0,95.00,0
1,116,18,2,28,2,1,1,0,1,1,0,5,0,0,0,61.00,0
2,11,18,7,25,1,2,1,0,1,2,1,5,0,0,0,129.75,1
3,3,17,9,12,0,1,1,0,1,2,0,5,0,0,0,152.00,3
4,28,18,3,7,1,3,1,0,1,2,0,4,0,0,0,87.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29015,2,18,11,21,1,2,1,0,1,1,0,5,0,0,0,100.67,1
29016,18,18,4,12,0,2,0,0,1,2,0,5,0,0,0,119.00,1
29017,8,17,9,29,0,3,1,0,1,1,0,3,0,0,0,65.00,0
29018,20,18,9,18,2,2,1,0,1,1,0,4,0,0,0,90.00,0


# Normalizing the dataframe

In [12]:
# REMOVED_COL = "BookingID"
PREDICT_COL = "DEATH_EVENT"
ENCODING = []

df = pd.read_csv(DATA_PATH)
# df = df.drop(REMOVED_COL, axis=1) # axis: 0 for row, 1 for column


for col in df.columns:
    if not all(isinstance(x, (int, float)) for x in df[col]):
        lst = np.unique(df[col].astype(str))
        if col == PREDICT_COL:
            for i in range(len(lst)):
                ENCODING.append((i, lst[i]))
        for i in range(len(lst)):
            df[col] = df[col].replace(lst[i], i)
        lst = []


target = df.pop(PREDICT_COL)
print(target)
print(ENCODING)
df

0      1
1      1
2      1
3      1
4      1
      ..
294    0
295    0
296    0
297    0
298    0
Name: DEATH_EVENT, Length: 299, dtype: int64
[]


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280


In [13]:
# Name: BookingStatus, Length: 29020, dtype: object
# dtype can't be object or will get error for normalizer(numeric_features.iloc[:3])
target = target.astype('int')
df = df.astype('int')
print(type(target))
target

<class 'pandas.core.series.Series'>


0      1
1      1
2      1
3      1
4      1
      ..
294    0
295    0
296    0
297    0
298    0
Name: DEATH_EVENT, Length: 299, dtype: int32

In [14]:
numeric_feature_names = [name for name in df.columns]
# numeric_feature_names = ['LeadTime', 'ArrivalYear', 'ArrivalMonth',  'ArrivalDate', 'NumWeekendNights', "NumWeekNights", "MealPlan", "Parking", "RoomType", "NumAdults", "NumChildren", "MarketSegment", "RepeatedGuest", "NumPrevCancellations", "NumPreviousNonCancelled", "AvgRoomPrice", "SpecialRequests"]
numeric_features = df[numeric_feature_names]
numeric_features.head()

print(type(numeric_features))

<class 'pandas.core.frame.DataFrame'>


In [15]:
# numeric_features = np.asarray(numeric_features).astype('float32')

tf.convert_to_tensor(numeric_features)


<tf.Tensor: shape=(299, 12), dtype=int32, numpy=
array([[  75,    0,  582, ...,    1,    0,    4],
       [  55,    0, 7861, ...,    1,    0,    6],
       [  65,    0,  146, ...,    1,    1,    7],
       ...,
       [  45,    0, 2060, ...,    0,    0,  278],
       [  45,    0, 2413, ...,    1,    1,  280],
       [  50,    0,  196, ...,    1,    1,  285]])>

In [16]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features.to_numpy())


In [17]:
numeric_features = pd.DataFrame(numeric_features)
type(numeric_features)


pandas.core.frame.DataFrame

In [18]:
normalizer(numeric_features.iloc[:3])


<tf.Tensor: shape=(3, 12), dtype=float32, numpy=
array([[ 1.1933013e+00, -8.7110478e-01,  1.6571533e-04, -8.4757942e-01,
        -1.5305597e+00,  1.3592716e+00,  1.6816784e-02,  1.8290954e-02,
        -1.5040385e+00,  7.3568821e-01, -6.8768191e-01, -1.6295023e+00],
       [-4.9089664e-01, -8.7110478e-01,  7.5146399e+00, -8.4757942e-01,
        -7.0769452e-03, -7.3568821e-01,  0.0000000e+00,  1.8290954e-02,
        -1.4197825e-01,  7.3568821e-01, -6.8768191e-01, -1.6036906e+00],
       [ 3.5120234e-01, -8.7110478e-01, -4.4993877e-01, -8.4757942e-01,
        -1.5305597e+00, -7.3568821e-01, -1.0380728e+00,  1.8290954e-02,
        -1.7310485e+00,  7.3568821e-01,  1.4541606e+00, -1.5907848e+00]],
      dtype=float32)>

In [19]:
print(normalizer)

In [20]:
len(numeric_features.iloc[0])

12

In [21]:
from math import floor
# some complex math
nodes = len(numeric_features)/(4 * (len(numeric_features.iloc[0]) + len(ENCODING)))
res = 0
for i in range(floor(nodes), 0, -1):
    if (i & (i - 1)) == 0:
        res = i
        break

lst = [res]
for i in range(3):
    res /= 2
    lst.append(res)
print(lst)


[4, 2.0, 1.0, 0.5]


In [22]:
from math import floor

# some complex math
nodes = len(numeric_features)/(2 * (len(numeric_features.iloc[0]) + len(ENCODING)))
res = floor(nodes/15)

print(res*1, res*2, res*4, res*8)


# for i in range(nodes, 0, -1):
#     if (i & (i - 1)) == 0:
#         res = i
#         break
#
# lst = [res]
# for i in range(4):
#     res /= 2
#     lst.append(res)
# print(lst)

0 0 0 0


In [23]:
def get_basic_model():
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(len(ENCODING), activation='sigmoid')
        ])

    model.compile(optimizer='adam',
                    loss="sparse_categorical_crossentropy", # tf.keras.losses.BinaryCrossentropy(from_logits=True)
                    metrics=['accuracy'])
    return model

get_basic_model().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 12)                25        
 on)                                                             
                                                                 
 dense (Dense)               (None, 128)               1664      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

In [24]:
print(numeric_features.iloc[:3])

   age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0   75        0                       582         0                 20   
1   55        0                      7861         0                 38   
2   65        0                       146         0                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1     265000                 1           130    1   
1                    0     263358                 1           136    1   
2                    0     162000                 1           129    1   

   smoking  time  
0        0     4  
1        0     6  
2        1     7  


In [25]:
model = get_basic_model()
history = model.fit(numeric_features.to_numpy(), target, epochs=30, batch_size=BATCH_SIZE)


Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'ArgMax_1' defined at (most recent call last):
    File "C:\Users\Kevin Tang\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Kevin Tang\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Kevin Tang\Documents\HackTheNorth2023\backend\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Kevin Tang\Documents\HackTheNorth2023\backend\venv\lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance
      app.start()
    File "C:\Users\Kevin Tang\Documents\HackTheNorth2023\backend\venv\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\Kevin Tang\Documents\HackTheNorth2023\backend\venv\lib\site-packages\torn

In [45]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(3):
  print(row)


(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([  10, 2018,    3,   31,    0,    1,    0,    0,    0,    1,    0,
          2,    0,    0,    0,   95,    0])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([ 116, 2018,    2,   28,    2,    1,    0,    0,    0,    1,    0,
          4,    0,    0,    0,   61,    0])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([  11, 2018,    7,   25,    1,    2,    0,    0,    0,    2,    1,
          4,    0,    0,    0,  129,    1])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [46]:
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()
history = model.fit(numeric_batches, epochs=90)

Epoch 1/90
29/29 [==============================] - 1s 3ms/step - loss: 0.5702 - accuracy: 0.6997
Epoch 2/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4738 - accuracy: 0.7826
Epoch 3/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4436 - accuracy: 0.8005
Epoch 4/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4265 - accuracy: 0.8103
Epoch 5/90
29/29 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8187
Epoch 6/90
29/29 [==============================] - 0s 2ms/step - loss: 0.4040 - accuracy: 0.8251
Epoch 7/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3958 - accuracy: 0.8281
Epoch 8/90
29/29 [==============================] - 0s 2ms/step - loss: 0.3841 - accuracy: 0.8326
Epoch 9/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3770 - accuracy: 0.8369
Epoch 10/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3723 - accuracy: 0.8387
Epoch 11/90
29/29 [

In [243]:
plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

# 💀💀💀💀💀💀💀💀💀

NameError: name 'plt' is not defined

In [47]:
# pd.DataFrame(numeric_features.numpy())
print(type(numeric_features))
print(numeric_features)
predictions = model.predict(np.array(numeric_features.iloc[:20]))

print("asdf")

lst = ["Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled"]
# print(predictions)

for i in range(len(predictions)-1):
    print(f"{predictions[i]}: {lst[i]}")
    if predictions[i][0] > predictions[i][1]:
        print("Predicted: cancled")
    else:
        print("Predicted: not_canceled")


<class 'pandas.core.frame.DataFrame'>
       LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  NumWeekendNights  \
0            10         2018             3           31                 0   
1           116         2018             2           28                 2   
2            11         2018             7           25                 1   
3             3         2017             9           12                 0   
4            28         2018             3            7                 1   
...         ...          ...           ...          ...               ...   
29015         2         2018            11           21                 1   
29016        18         2018             4           12                 0   
29017         8         2017             9           29                 0   
29018        20         2018             9           18                 2   
29019       274         2018             7           31                 2   

       NumWeekNights  MealPlan  Parki

In [11]:
pwd

'C:\\Users\\Kevin Tang\\Documents\\HackTheNorth2023\\AI'

In [63]:
DATA_PATH = "dataset/updated_test_data.csv"

# prediction
REMOVED_COL = "BookingID"
PREDICT_COL = "BookingStatus"

df = pd.read_csv(DATA_PATH)
df = df.drop(REMOVED_COL, axis=1) # axis: 0 for row, 1 for column
# target = df.drop(PREDICT_COL)

for col in df.columns:
    if not all(isinstance(x, (int, float)) for x in df[col]):
        lst = np.unique(df[col].astype(str))
        for i in range(len(lst)):
            df[col] = df[col].replace(lst[i], i)


# target = df.drop(PREDICT_COL)
print(len(numeric_feature_names))
df = df.astype('int')
numeric_feature_names = [name for name in df.columns]
print(len(numeric_feature_names))

numeric_features = df[numeric_feature_names]
numeric_features.head()
tf.convert_to_tensor(numeric_features)
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features.to_numpy())


# new_model = tf.keras.models.load_model('../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_ac699aad-290c-47bc-8f7c-a80eb84b0b01/model.keras')

new_model = tf.keras.models.load_model('../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_ac699aad-290c-47bc-8f7c-a80eb84b0b01/model')

# new_model.load_weights('../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_ac699aad-290c-47bc-8f7c-a80eb84b0b01/model')

predictions = new_model.predict(np.array(numeric_features))



17
17
227/227 [==============================] - 0s 549us/step


In [58]:
print(predictions)

[[0.15406722 0.9829524 ]
 [0.16708525 0.9810974 ]
 [0.02197331 0.99994904]
 ...
 [0.0456804  0.99965805]
 [0.07709794 0.99846077]
 [0.09470014 0.9967107 ]]
